<a href="https://colab.research.google.com/github/shreyasathyanarayana1/BIS-Lab/blob/main/BIS_Lab_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random

class ACO_Network:
    def __init__(self, nodes, edges, alpha=1, beta=2, evaporation_rate=0.5, ants=10, iterations=50):
        self.nodes = nodes
        self.edges = edges  # {(node1, node2): [delay, capacity]}
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.ants = ants
        self.iterations = iterations
        self.pheromone = {edge: 1.0 for edge in edges}  # initial pheromone

    def heuristic(self, edge):
        delay, capacity = self.edges[edge]
        return 1 / (delay * (1 + capacity))  # smaller delay & congestion -> higher heuristic

    def choose_next_node(self, current, visited):
        neighbors = [n for n in self.nodes if (current, n) in self.edges and n not in visited]
        if not neighbors:
            return None
        probabilities = []
        for neighbor in neighbors:
            edge = (current, neighbor)
            tau = self.pheromone[edge] ** self.alpha
            eta = self.heuristic(edge) ** self.beta
            probabilities.append(tau * eta)
        total = sum(probabilities)
        probabilities = [p / total for p in probabilities]
        next_node = random.choices(neighbors, probabilities)[0]
        return next_node

    def route_ant(self, start, end):
        path = [start]
        visited = set(path)
        while path[-1] != end:
            next_node = self.choose_next_node(path[-1], visited)
            if not next_node:  # dead end
                return None, float('inf')
            path.append(next_node)
            visited.add(next_node)
        cost = sum(self.edges[(path[i], path[i+1])][0] * (1 + self.edges[(path[i], path[i+1])][1])
                   for i in range(len(path)-1))
        return path, cost

    def update_pheromones(self, all_paths):
        # Evaporation
        for edge in self.pheromone:
            self.pheromone[edge] *= (1 - self.evaporation_rate)
        # Deposit
        for path, cost in all_paths:
            if path is None:
                continue
            for i in range(len(path)-1):
                self.pheromone[(path[i], path[i+1])] += 1.0 / cost

    def run(self, start, end):
        best_path = None
        best_cost = float('inf')
        for _ in range(self.iterations):
            all_paths = [self.route_ant(start, end) for _ in range(self.ants)]
            self.update_pheromones(all_paths)
            for path, cost in all_paths:
                if cost < best_cost:
                    best_cost = cost
                    best_path = path
        return best_path, best_cost

# ---- User Input ----
nodes = input("Enter nodes separated by comma (e.g., A,B,C,D): ").split(",")
edges = {}
print("Enter edges in format: node1 node2 delay capacity (enter 'done' to finish)")
while True:
    data = input()
    if data.lower() == "done":
        break
    n1, n2, delay, capacity = data.split()
    edges[(n1, n2)] = [float(delay), float(capacity)]
    edges[(n2, n1)] = [float(delay), float(capacity)]  # undirected network

start = input("Enter start node: ")
end = input("Enter end node: ")

aco = ACO_Network(nodes, edges)
best_path, best_cost = aco.run(start, end)
print(f"Optimized path: {best_path} with cost (delay + congestion): {best_cost:.2f}")
